In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# Prepare paths:
import glob
from pathlib import Path
inpath = '../input/indoor-location-navigation/'
metapath = inpath + 'metadata/'
trainpath = inpath + 'train/'
testpath = inpath + 'test/'

# Extract testing files, buildings and sites:
os.system(f'grep SiteID {testpath}/* > test_buildings.txt' )
test_buildings = pd.read_csv('test_buildings.txt',sep='\t',header=None,names=['file','building','site'])
test_buildings['file'] = test_buildings['file'].apply(lambda x: x[:-2])
test_buildings['building'] = test_buildings['building'].apply(lambda x: x[7:])

# How many buildings in the testing set?
buildings = np.unique(test_buildings['building'])
print('There are',len(buildings),'buildings in the testing set.')

test_buildings.head()


There are 24 buildings in the testing set.


,file,building,site
0,../input/indoor-location-navigation/test//00ff...,5da1389e4db8ce0c98bd0547,SiteName:和达城商场
1,../input/indoor-location-navigation/test//01c4...,5da138b74db8ce0c98bd4774,SiteName:万象城
2,../input/indoor-location-navigation/test//030b...,5da138764db8ce0c98bcaa46,SiteName:银泰百货
3,../input/indoor-location-navigation/test//0389...,5dbc1d84c1eb61796cf7c010,SiteName:杭州大悦城
4,../input/indoor-location-navigation/test//0402...,5da1383b4db8ce0c98bc11ab,SiteName:永旺梦乐城


In [3]:
# Compile C++ pre-processing code:
er=os.system("g++ ../input/indoor-cpp/1_preprocess.cpp -std=c++11 -o preprocess")
if(er): print("Error")

# Reformat the testing set:
os.system('mkdir test')
for i,(path_filename,building) in enumerate(zip(test_buildings['file'],test_buildings['building'])):
    er=os.system(f'./preprocess {path_filename} test {building} {0}') #since we do not know the floor, I put 0.
    if(er): print("Error:",path_filename)

In [4]:
# Acceleration, magnetic and orientation testing data:
os.system('mkdir indoor_testing_accel')
os.system("g++ ../input/indoor-cpp/2_preprocess_accel.cpp -std=c++11 -o preprocess_accel")
for building in buildings:
    os.system(f'./preprocess_accel {building}')

In [5]:
# Wifi testing data:
os.system('mkdir test_wifi')
os.system("g++ /kaggle/input/indoor-cpp/2_preprocess_wifi.cpp -std=c++11 -o preprocess_wifi")
for building in buildings:
    os.system(f'./preprocess_wifi {building}')